In [ ]:
# what happens if we use R????

In [ ]:
getwd()

In [ ]:
install.packages('docopt')
install.packages('git2r')

In [ ]:
install.packages('styler')

In [ ]:
source('../src/untitled.r')

In [ ]:
print_x('boo')

In [ ]:
print('hi')

In [ ]:
library(data.table) 

In [ ]:
#library(

In [ ]:
import pandas as pd  # this doesnt work cos its python.

In [ ]:
# hello hello hello hello hello hello more than 88 characters what happpens hellloooooooooooooooooooooooooooooo

In [ ]:
library(bigrquery)

In [ ]:
library(bigQueryR)

In [ ]:
#https://www.infoworld.com/article/3622926/how-to-use-r-with-bigquery.html

In [ ]:
library(bigrquery)

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-data-prod',
    dataset = 'fintrans_visa'
    )

In [ ]:
dbListTables(con)

In [ ]:
con <- dbConnect(
    bigquery(),
    project = 'ons-fintrans-analysis-prod',
    dataset = 'fin_wip_notebook'
    )

In [ ]:
dbListTables(con)

In [ ]:
library(dplyr)

In [ ]:
sml <- tbl(con, "sml_cardholders_q")

In [ ]:
str(sml)

In [ ]:
glimpse(sml)

In [ ]:
df <- select(sml, date_time) %>% distinct() %>% collect()

# need to add collect to view things

In [ ]:
df

In [ ]:
select(sml, date_time) %>% distinct() %>% show_query()

In [ ]:
# or running SQL From within R
sql <- "SELECT * FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location LIMIT 10"
library(DBI)
my_results <- dbGetQuery(con, sql)

In [ ]:
my_results